# Assignment 3, Part 2: T5 SQuAD Model 

Welcome to the part 2 of testing the models for this week's assignment. This time we will perform decoding using the T5 SQuAD model. In this notebook we'll perform Question Answering by providing a "Question", its "Context" and see how well we get the "Target" answer. 

## IMPORTANT

- As you cannot save the changes you make to this colab, you have to make a copy of this notebook in your own drive and run that. You can do so by going to `File -> Save a copy in Drive`. Close this colab and open the copy which you have made in your own drive.

- Go to this [google drive folder](https://drive.google.com/drive/folders/1rOZsbEzcpMRVvgrRULRh1JPFpkIG_JOz?usp=sharing) named `NLP C4 W3 Colabs & Data`. In the folder, next to its name use the drop down menu to select `"Add shortcut to Drive" -> "My Drive" and then press ADD SHORTCUT`. This should add a shortcut to the folder `NLP C4 W3 Colabs & Data` within your own google drive. Please make sure this happens, as you'll be reading the data for this notebook from this folder.

- Make sure your runtime is GPU (_not_ CPU or TPU). And if it is an option, make sure you are using _Python 3_. You can select these settings by going to `Runtime -> Change runtime type -> Select the above mentioned settings and then press SAVE`

**Note: Restarting the runtime maybe required**.

Colab will tell you if the restarting is necessary -- you can do this from the:

Runtime > Restart Runtime

option in the dropdown.

## Outline

- [Part 0: Downloading and loading dependencies](#0)
- [Part 1: Mounting your drive for data accessibility](#1)
- [Part 2: Getting things ready](#2)
- [Part 3: Fine-tuning on SQuAD](#3)
    - [3.1 Loading in the data and preprocessing](#3.1)
    - [3.2 Decoding from a fine-tuned model](#3.2)

<a name='0'></a>
# Part 0: Downloading and loading dependencies

Uncomment the code cell below and run it to download some dependencies that you will need. You need to download them once every time you open the colab. You can ignore the `kfac` error.

In [1]:
!pip -q install trax==1.3.4

     |████████████████████████████████| 368kB 4.5MB/s 
     |████████████████████████████████| 2.6MB 19.2MB/s 
     |████████████████████████████████| 163kB 42.9MB/s 
     |████████████████████████████████| 1.5MB 28.3MB/s 
     |████████████████████████████████| 1.1MB 55.9MB/s 
     |████████████████████████████████| 3.5MB 54.0MB/s 
     |████████████████████████████████| 1.0MB 55.2MB/s 
     |████████████████████████████████| 307kB 56.2MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 655kB 54.9MB/s 
     |████████████████████████████████| 5.3MB 18.2MB/s 
     |████████████████████████████████| 368kB 51.6MB/s 
     |████████████████████████████████| 983kB 54.9MB/s 
     |████████████████████████████████| 194kB 56.3MB/s 
     |████████████████████████████████| 358kB 52.7MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 890kB 53.0MB/s 
     |████████████████████████████████| 3.0MB 49.4M

In [2]:
import string
import t5
import numpy as np
import trax 
from trax.supervised import decoding
import textwrap 
# Will come handy later.
wrapper = textwrap.TextWrapper(width=70)

INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 


<a name='1'></a>
# Part 1: Mounting your drive for data accessibility

Run the code cell below and follow the instructions to mount your drive. The data is the same as used in the coursera version of the assignment.

In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [7]:
import os
os.chdir('/content/drive/My Drive/')

In [8]:
ls

 0117_Sherman_Template_SlidesMania.gslides
 2013-12.pptx
 50YearsDataScience.gdoc
 activity-report.xls.gsheet
'AFFIDAVIT OF LOSS.docx'
'Airtasker - Verne.gdoc'
'Alexa Sept.pptx'
'Alexa Traffic Rank Sept.xlsm'
 all_buildings_meta_data.gsheet
 AM.pdf
 API_SP.URB.TOTL_DS2_en_csv_v2_247888/
 application-e20200201.doc
'Application Form_06.11.2015.gsheet'
'Application Form (140116).xlsx'
'Application Form (140116).xlsx.gsheet'
'Artificial Intelligence for Marketing.pdf'
'Asian Profit Calculator - Mark Agbulos Beta-ver5.xlsx'
'Assignment for Junior Developer Candidates (Manila)_Feb 2014.pdf'
'Assignment tracker.gsheet'
 B2B.gslides
'Balanced_Scorecard - OngtecoV.xlsx'
'BB Reference.gsheet'
'BCG InsideSherpa Core Strategy - Telco (Task 2 Additional Data)-update.xlsx'
'B Milestone.gslides'
'Bollinger Bonds.pdf'
'brutti ex2.docx'
'brutti ex2.gdoc'
'btb centris-2.jpg'
'BTI Employment Application Form - 2016.xlsx'
'Business Roadmap (1).gslides'
'Business Roadmap.gslides'
'Campaign Data 2.ipynb'
 c

<a name='2'></a>
# Part 2: Getting things ready 

Run the code cell below to ready some functions which will later help us in decoding. The code and the functions are the same as the ones you previsouly ran on the coursera version of the assignment.

In [9]:
# Reminder: need to change dir to 'NLP C4 W3 Colabs & Data' instead of NLP C4 W3 Data.

PAD, EOS, UNK = 0, 1, 2

def detokenize(np_array):
  return trax.data.detokenize(
      np_array,
      vocab_type='sentencepiece',
      vocab_file='sentencepiece.model',
      vocab_dir='/content/drive/My Drive/NLP C4 W3 Colabs & Data/')

def tokenize(s):
  # The trax.data.tokenize function operates on streams,
  # that's why we have to create 1-element stream with iter
  # and later retrieve the result with next.
  return next(trax.data.tokenize(
      iter([s]),
      vocab_type='sentencepiece',
      vocab_file='sentencepiece.model',
      vocab_dir='/content/drive/My Drive/NLP C4 W3 Colabs & Data/'))
 
vocab_size = trax.data.vocab_size(
    vocab_type='sentencepiece',
    vocab_file='sentencepiece.model',
    vocab_dir='/content/drive/My Drive/NLP C4 W3 Colabs & Data/')

def get_sentinels(vocab_size):
    sentinels = {}

    for i, char in enumerate(reversed(string.ascii_letters), 1):

        decoded_text = detokenize([vocab_size - i]) 
        
        # Sentinels, ex: <Z> - <a>
        sentinels[decoded_text] = f'<{char}>'
        
    return sentinels

sentinels = get_sentinels(vocab_size)    


def pretty_decode(encoded_str_list, sentinels=sentinels):
    # If already a string, just do the replacements.
    if isinstance(encoded_str_list, (str, bytes)):
        for token, char in sentinels.items():
            encoded_str_list = encoded_str_list.replace(token, char)
        return encoded_str_list
  
    # We need to decode and then prettyfy it.
    return pretty_decode(detokenize(encoded_str_list))    

<a name='3'></a>
# Part 3: Fine-tuning on SQuAD

Now let's try to fine tune on SQuAD and see what becomes of the model. For this, we need to write a function that will create and process the SQuAD `tf.data.Dataset`. Below is how T5 pre-processes SQuAD dataset as a text2text example. Before we jump in, we will have to first load in the data. 

<a name='3.1'></a>
### 3.1 Loading in the data and preprocessing

You first start by loading in the dataset. The text2text example for a SQuAD example looks like:

```
{
  'inputs': 'question: <question> context: <article>',
  'targets': '<answer_0>',
}
```

The squad pre-processing function takes in the dataset and processes it using the sentencePiece vocabulary you have seen above. It generates the features from the vocab and encodes the string features. It takes on question, context, and answer, and returns "question: Q context: C" as input and "A" as target.

In [10]:
# Retrieve Question, C, A and return "question: Q context: C" as input and "A" as target.
def squad_preprocess_fn(dataset, mode='train'):
  return t5.data.preprocessors.squad(dataset)

In [11]:
# train generator, this takes about 1 minute
train_generator_fn, eval_generator_fn = trax.data.tf_inputs.data_streams(
  'squad/v1.1:2.0.0',
  data_dir='/content/drive/My Drive/NLP C4 W3 Colabs & Data/data/',
  bare_preprocess_fn=squad_preprocess_fn,
  input_name='inputs',
  target_name='targets'
)
train_generator = train_generator_fn()
next(train_generator)

(b'question: When did Cook release his version of The Time of My Life ? context: The American Idol Songwriter contest was also held this season . From ten of the most popular submissions , each of the final two contestants chose a song to perform , although neither of their selections was used as the " coronation song " . The winning song , " The Time of My Life " , was recorded by David Cook and released on May 22 , 2008 . ',
 b'May 22 , 2008')

In [12]:
#print example from train_generator
(inp, out) = next(train_generator)
print(inp.decode('utf8').split('context:')[0])
print()
print('context:', inp.decode('utf8').split('context:')[1])
print()
print('target:', out.decode('utf8'))

question: What was the first state to formally secede ? 

context:  Sectional tensions had long existed between the states located north of the Mason – Dixon line and those south of it , primarily centered on the " peculiar institution " of slavery and the ability of states to overrule the decisions of the national government . During the 1840s and 1850s , conflicts between the two sides became progressively more violent . After the election of Abraham Lincoln in 1860 ( who southerners thought would work to end slavery ) states in the South seceded from the United States , beginning with South Carolina in late 1860 . On April 12 , 1861 , forces of the South ( known as the Confederate States of America or simply the Confederacy ) opened fire on Fort Sumter , whose garrison was loyal to the Union . 

target: South Carolina


<a name='3.2'></a>
### 3.2 Decoding from a fine-tuned model

You will now use an existing model that we trained for you. You will initialize, then load in your model, and then try with your own input. 

In [13]:
# Initialize the model 
model = trax.models.Transformer(
    d_ff = 4096,
    d_model = 1024,
    max_len = 2048,
    n_heads = 16,
    dropout = 0.1,
    input_vocab_size = 32000,
    n_encoder_layers = 24,
    n_decoder_layers = 24,
    mode='predict')  # Change to 'eval' for slow decoding.

In [14]:
# load in the model
# this will take a minute
shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)
model.init_from_file('/content/drive/My Drive/NLP C4 W3 Colabs & Data/models/model_squad.pkl.gz',
                     weights_only=True, input_signature=(shape11, shape11))

In [15]:
# Uncomment to see the transformer's structure.
print(model)

Serial_in2_out2[
  Select[0,1,1]_in2_out3
  Branch_out2[
    []
    PaddingMask(0)
  ]
  Cache_in2_out2[
    Serial_in2_out2[
      Embedding_32000_1024
      Dropout
      PositionalEncoding
      Serial_in2_out2[
        Branch_in2_out3[
          None
          Serial_in2_out2[
            LayerNorm
            Serial_in2_out2[
              Dup_out2
              Dup_out2
              Serial_in4_out2[
                Parallel_in3_out3[
                  Dense_1024
                  Dense_1024
                  Dense_1024
                ]
                PureAttention_in4_out2
                Dense_1024
              ]
            ]
            Dropout
          ]
        ]
        Add_in2
      ]
      Serial[
        Branch_out2[
          None
          Serial[
            LayerNorm
            Dense_4096
            Relu
            Dropout
            Dense_1024
            Dropout
          ]
        ]
        Add_in2
      ]
      Serial_in2_out2[
        Branch_in2_out3[
 

In [16]:
# create inputs
# a simple example 
# inputs = 'question: She asked him where is john? context: John was at the game'

# an extensive example
inputs = 'question: What are some of the colours of a rose? context: A rose is a woody perennial flowering plant of the genus Rosa, in the family Rosaceae, or the flower it bears.There are over three hundred species and tens of thousands of cultivars. They form a group of plants that can be erect shrubs, climbing, or trailing, with stems that are often armed with sharp prickles. Flowers vary in size and shape and are usually large and showy, in colours ranging from white through yellows and reds. Most species are native to Asia, with smaller numbers native to Europe, North America, and northwestern Africa. Species, cultivars and hybrids are all widely grown for their beauty and often are fragrant.'

In [17]:
# tokenizing the input so we could feed it for decoding
print(tokenize(inputs))
test_inputs = tokenize(inputs) 

[  822    10   363    33   128    13     8  6548    13     3     9  4659
    58  2625    10    71  4659    19     3     9  1679    63 24999  5624
    53  1475    13     8     3   729   302 15641     6    16     8   384
 15641  8433    15     6    42     8  5624    34  4595     7     5  7238
    33   147   386  6189  3244    11     3   324     7    13  2909    13
 10357   291     7     5   328   607     3     9   563    13  2677    24
    54    36     3    15 12621 21675     7     6 11908     6    42  5032
    53     6    28  6269     7    24    33   557     3  8715    28  4816
     3  2246 19376     7     5 20294  5215    16   812    11  2346    11
    33  1086   508    11   504    63     6    16  6548     3  6836    45
   872   190  4459     7    11  1131     7     5  1377  3244    33  4262
    12  3826     6    28  2755  2302  4262    12  1740     6  1117  1371
     6    11  3457 24411  2648     5     3  7727   725     6 10357   291
     7    11  9279     7    33    66  5456  4503   

In [18]:
# Temperature is a parameter for sampling.
#   # * 0.0: same as argmax, always pick the most probable token
#   # * 1.0: sampling from the distribution (can sometimes say random things)
#   # * values inbetween can trade off diversity and quality, try it out!
output = decoding.autoregressive_sample(model, inputs=np.array(test_inputs)[None, :],
                                        temperature=0.0, max_length=10)
print(wrapper.fill(pretty_decode(output[0])))

white through yellows and reds


### Note: As you can see the RAM is almost full, it is because the model and the decoding is memory heavy. You can run decoding just once. Running it the second time with another example might give you the same answer as before, or not run at all (crash). If that happens restart the runtime (see how to at the start of the notebook) and run all the cells again.